In [1]:
import pandas as pd
import os
import hashlib
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
file_path="~/SageMaker/Rain_Estimation_Using_Loosely_Matched_Stations/CMLs_20110609_20110813.dat"

In [3]:
df = pd.read_table(file_path, sep="\s+", index_col=False)

In [4]:
df["Start_Concat"]=df["YStart"].astype(str)+" "+df["XStart"].astype(str)
df["End_Concat"]=df["YEnd"].astype(str)+" "+df["XEnd"].astype(str)

In [5]:
def convert_to_four_letter_hash(concat):
    h = hashlib.sha1(str.encode(concat))
    d = h.digest()
    s = ""
    for i in range(0,4):
        x = d[i] % 52
        if x >= 26:
            s += chr(ord('A') + x - 26)
        else:
            s += chr(ord('a') + x)
    return s

In [6]:
print(f'{convert_to_four_letter_hash("51.80485 5.11687")}-{convert_to_four_letter_hash("51.83246 5.08996")}')


XmQw-fTxW


In [7]:
def convert_datetime_to_time(datetime):
    yyyy=datetime[:4]
    mm=datetime[4:6]
    dd=datetime[6:8]
    hh=datetime[-4:-2]
    ss=datetime[-2:]
    
    return(f"{dd}-{mm}-{yyyy} {hh}:{ss}")
    

In [8]:
df["ID"]=df["Start_Concat"].apply(convert_to_four_letter_hash).astype(str)+"-"+df["End_Concat"].apply(convert_to_four_letter_hash).astype(str)

In [9]:
df["Time"]=df["DateTime"].astype(str).apply(convert_datetime_to_time)

In [10]:
ids=df.ID.unique()

In [11]:
df["PowerTLTMmax[dBm]"]=5
df["PowerTLTMmin[dBm]"]=4.8
df.rename(columns={"Pmin": "PowerRLTMmin[dBm]", "Pmax": "PowerRLTMmax[dBm]"}, inplace=True)
df.dropna(inplace=True)

In [16]:
df[df.ID=="XJHB-jwZI"]

,Frequency,DateTime,PowerRLTMmin[dBm],PowerRLTMmax[dBm],PathLength,XStart,YStart,XEnd,YEnd,ID,Start_Concat,End_Concat,Time,PowerTLTMmax[dBm],PowerTLTMmin[dBm]
0,39.263,201106090815,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 08:15,5,4.8
5410,39.263,201106090830,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 08:30,5,4.8
8347,39.263,201106090845,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 08:45,5,4.8
9709,39.263,201106090900,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 09:00,5,4.8
12958,39.263,201106090915,-61,-59,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,09-06-2011 09:15,5,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451707,39.263,201108081245,-61,-60,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,08-08-2011 12:45,5,4.8
1454381,39.263,201108081300,-61,-60,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,08-08-2011 13:00,5,4.8
1456428,39.263,201108081315,-61,-60,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,08-08-2011 13:15,5,4.8
1457720,39.263,201108081330,-61,-60,3.12961,4.75907,52.41104,4.7889,52.38963,XJHB-jwZI,52.41104 4.75907,52.38963 4.7889,08-08-2011 13:30,5,4.8


In [14]:
root="~/SageMaker/Rain_Estimation_Using_Loosely_Matched_Stations/CellEnMon/datasets/dme/09062011_13082011/raw2"
columns=["Time","PowerTLTMmax[dBm]","PowerTLTMmin[dBm]","PowerRLTMmax[dBm]","PowerRLTMmin[dBm]"]
for id in ids[3084:]:
    T,R=id.split("-")
    d=df[df.ID==id]
    d.to_csv(f'{root}/{T}_{d.YStart.iloc[0]}_{d.XStart.iloc[0]}_{R}_{d.YEnd.iloc[0]}_{d.XEnd.iloc[0]}.csv',index=False, columns=columns)
    

In [ ]:
df